# Lab 4 - Three ways for Agents to Interact

The 3 models:

1. Separate calls to Runner.run()

2. Agent Handoffs, turns within a Runner.run()

    A -> B -> C

3. Agents as Tools, turns within a Runner.run()

    A -> B -> A


Approach (1) is the least juicy & fashionable, but it's the one I recommend the most!

Approach (2) seems really hard to make reliable.

Approach (3) works OK with good enough prompts..

In [ ]:
from agents import Agent, Runner, trace, handoff
from IPython.display import Markdown, display
from pydantic import BaseModel, Field
from dotenv import load_dotenv
load_dotenv(override=True)


In [7]:

entrepreneur_instructions = "You generate novel, exciting business ideas for Autonomous AI Agents"
entrepreneur_input = """
Propose one compelling business idea for applying Autonomous AI Agents to any industry you choose.
The idea should be unique, high potential commercial impact, and make true use of the autonomous capabilities of AI Agents.
"""

evaluator_instructions = "You evaluate novel, exciting business ideas for Autonomous AI Agents"
evaluator_input = "Evaluate the following business idea; assess it for uniqueness, potential commercial impact, and use of Autonomous AI Agents:\n\n"

## Now let's define 2 important Pydantic Objects to describe our Data

In [8]:
class Idea(BaseModel):
    title: str = Field(description="The title of the idea")
    description: str = Field(description="A detailed description of the idea, in Markdown")

    def print_summary(self):
        print(f"Title: {self.title}")
        display(Markdown(self.description))

In [9]:
class Evaluation(BaseModel):
    idea: Idea = Field(description="The idea that you are evaluating")
    uniqueness_feedback: str = Field(description="Your view on how unique the idea is")
    uniqueness_score: int = Field(description="How unique the idea is, on a scale of 1 to 10, where 1 is mundane and 10 is groundbreaking")
    commercial_feedback: str = Field(description="Your view on the commercial potential of the idea")
    commercial_score: int = Field(description="How high the commercial potential of the idea is from 1 to 10, where 1 is barely profitable and 10 is a billion dollar idea")
    autonomy_feedback: str = Field(description="Your commentary on to what extent the idea truly benefits from the autonomous nature of AI Agents")
    autonomy_score: int = Field(description="How deeply the idea involves autonomous agents from 1 to 10, where 1 is that it plays no meaningful role, and 10 is that it is pivotal")

    def is_brilliant(self):
        return self.uniqueness_score > 8 and self.commercial_score > 8 and self.autonomy_score > 8
    
    def print_summary(self):
        print(f"Idea: {self.idea.title}")
        print(f"Uniqueness: {self.uniqueness_score}/10: ({self.uniqueness_feedback})")
        print(f"Commercial: {self.commercial_score}/10: ({self.commercial_feedback})")
        print(f"Autonomy: {self.autonomy_score}/10: ({self.autonomy_feedback})")

## Approach 1 (Highly Recommended!) - Call Runner.run() separately

In [ ]:
entrepreneur = Agent(name="Entrepreneur", model="gpt-4.1-mini", instructions=entrepreneur_instructions, output_type=Idea)
result = await Runner.run(entrepreneur, entrepreneur_input)
idea = result.final_output_as(Idea)
idea.print_summary()

In [ ]:
evaluator = Agent(name="Evaluator", model="gpt-4.1-mini", instructions=evaluator_instructions, output_type=Evaluation)
result = await Runner.run(evaluator, f"{evaluator_input}{idea}")
evaluation = result.final_output_as(Evaluation)
evaluation.print_summary()

In [12]:
entrepreneur_latest = f"You were originally given the following task:\n\n{entrepreneur_input}\n\n"
remaining_attempts = 4

In [ ]:
with trace("Entrepreneur Approach 1"):
    while not evaluation.is_brilliant() and remaining_attempts:
        remaining_attempts -= 1
        entrepreneur_latest += f"You responded with this idea:\n\n{idea}\n\nYou received this feedback:\n\n{evaluation}\n\n"
        entrepreneur_latest += "Now respond with an improved idea that adresses the feedback. Do not directly reference the feedback.\n\n"
        result = await Runner.run(entrepreneur, entrepreneur_latest)
        idea = result.final_output_as(Idea)
        idea.print_summary()
        result = await Runner.run(evaluator, f"{evaluator_input}{idea}")
        evaluation = result.final_output_as(Evaluation)
        evaluation.print_summary()

## Approach 2 - Handoff

This seems to be the more unpredictable of the approaches, and I struggle to get it to ever handoff more than 1 loop..

In [14]:
entrepreneur_instructions = "You generate novel, exciting business ideas for Autonomous AI Agents that you pass to the Evaluator to evaluate"
entrepreneur_input = """
Propose one compelling business idea for applying Autonomous AI Agents to any industry you choose.
The idea should be unique, high potential commercial impact, and make true use of the autonomous capabilities of AI Agents.
You MUST handoff your idea to the Evaluator to be evaluated by using the tool Handoff_to_Evaluator. You must always call this tool with your idea or an improved idea.
"""

evaluator_instructions = """
You evaluate novel, exciting business ideas for Autonomous AI Agents.
Evaluate the business idea that you are given; assess it for uniqueness, potential commercial impact, and use of Autonomous AI Agents.
If you would not score the idea 9/10 or 10/10 in all 3 categories, then you must handoff the idea back to the Entrepreneur to improve it by using the tool Handoff_to_Entrepreneur and providing your feedback.
if you do score the idea 9/10 or better in all categories, then respond with the evaluation.
"""

In [15]:
entrepreneur = Agent(name="Entrepreneur", model="gpt-4.1-mini", instructions=entrepreneur_instructions, output_type=Idea)
evaluator = Agent(name="Evaluator", model="gpt-4.1-mini", instructions=evaluator_instructions, output_type=Evaluation)


def on_handoff(ctx, input):
    print("Handoff called")

entrepreneur.handoffs = [handoff(
    agent=evaluator,
    input_type=Idea,
    on_handoff=on_handoff,
    tool_name_override="Handoff_to_Evaluator",
    tool_description_override="Use this tool to handoff your idea to the Evaluator to evaluate. You must always call this tool."
)]

evaluator.handoffs = [handoff(
    agent=entrepreneur,
    input_type=Evaluation,
    on_handoff=on_handoff,
    tool_name_override="Handoff_to_Entrepreneur",
    tool_description_override="Use this tool to handoff your feedback to the Entrepreneur to improve the idea. You should call this tool if your feedback is less than 9/10 in any category."
)] 

In [ ]:
with trace("Entrepreneur Approach 2"):
    result = await Runner.run(entrepreneur, entrepreneur_input, max_turns=10)

In [ ]:
result.final_output.print_summary()

## Approach 3 - Agents as Tools

In [18]:

entrepreneur_instructions = "You generate novel, exciting business ideas for Autonomous AI Agents"
entrepreneur_input = """
Propose one compelling business idea for applying Autonomous AI Agents to any industry you choose.
The idea should be unique, high potential commercial impact, and make true use of the autonomous capabilities of AI Agents.
You must call the Evaluator tool to Evaluate the idea. If the idea is not 9 or 10 in all 3 scores,
then you must improve the idea and call the Evaluator tool again.
Keep improving the idea and calling the Evaluator tool until you get a 9 or 10 in all 3 scores.
Then finally respond with the idea.
"""
evaluator_instructions = """
You evaluate novel, exciting business ideas for Autonomous AI Agents.
Evaluate the following business idea; assess it for uniqueness, potential commercial impact, and use of Autonomous AI Agents.
"""



In [19]:
evaluator = Agent(name="Evaluator", model="gpt-4.1-mini", instructions=evaluator_instructions, output_type=Evaluation)
evaluator_tool = evaluator.as_tool(
    tool_name="Evaluator_Idea_Evaluator",
    tool_description="Evaluates the idea for uniqueness, potential commercial impact, and use of Autonomous AI Agents",
)

entrepreneur = Agent(name="Entrepreneur", model="gpt-4.1-mini", instructions=entrepreneur_instructions, output_type=Idea, tools=[evaluator_tool])

In [ ]:
with trace("Entrepreneur Approach 3"):
    result = await Runner.run(entrepreneur, entrepreneur_input)
    idea = result.final_output_as(Idea)
    idea.print_summary()

## OpenAI Traces

We've been writing all this information to OpenAI traces, the 'observability' framework built into OpenAI.

You can take a look at these traces here:

https://platform.openai.com/traces

There's also a nice way to visualize:

In [ ]:
from agents.extensions.visualization import draw_graph
draw_graph(entrepreneur)
